In [ ]:
import re
import math
import itertools
import numpy as np

In [ ]:
documents = [
     'This is the first-document.',
     'This document is the second document.',
     'And this is the third one.',
     'Is this the first document?'
]

documents = [document.lower() for document in documents]
documents = [re.sub('[^A-Za-z0-9\ ]+', ' ', document) for document in documents]

vocab = [document.split() for document in documents]
vocab = list(set(list(itertools.chain(*vocab))))
vocab = sorted(vocab)

print(f"documents: {documents}")
print(f"vocab: {vocab}")
print(f"number of documents: {len(documents)} \nvocab length: {len(vocab)}")

documents: ['this is the first document ', 'this document is the second document ', 'and this is the third one ', 'is this the first document ']
vocab: ['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']
number of documents: 4 
vocab length: 9


## Vector Normalization
Source : [machinelearningmastery](https://machinelearningmastery.com/vector-norms-machine-learning/)

1.   The L1 norm that is calculated as the sum of the absolute values of the vector.
2.   The L2 norm that is calculated as the square root of the sum of the squared vector values.
3.   The max norm that is calculated as the maximum vector values.



## TF-IDF
Source : [medium](https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089) by [William Scott](https://williamscott701.medium.com/)

    TF-IDF = Term Frequency (TF) * Inverse Document Frequency (IDF)

### Term Frequency
This measures the frequency of a word in a document. This highly depends on the length of the document and the generality of the word, for example, a very common word such as “was” can appear multiple times in a document. But if we take two documents with 100 words and 10,000 words respectively, there is a high probability that the common word “was” is present more in the 10,000 worded document. But we cannot say that the longer document is more important than the shorter document. For this exact reason, we perform normalization on the frequency value, we divide the frequency with the total number of words in the document.

Recall that we need to finally vectorize the document. When we plan to vectorize documents, we cannot just consider the words that are present in that particular document. If we do that, then the vector length will be different for both the documents, and it will not be feasible to compute the similarity. So, what we do is that we vectorize the documents on the vocab. Vocab is the list of all possible worlds in the corpus.

We need the word counts of all the vocab words and the length of the document to compute TF. In case the term doesn’t exist in a particular document, that particular TF value will be 0 for that particular document. In an extreme case, if all the words in the document are the same, then TF will be 1. The final value of the normalised TF value will be in the range of [0 to 1]. 0, 1 inclusive.



    tf(t,d) = count of t in d / number of words in d


#### Document Frequency

This measures the importance of documents in a whole set of the corpus. This is very similar to TF but the only difference is that TF is the frequency counter for a term t in document d, whereas DF is the count of occurrences of term t in the document set N. In other words, DF is the number of documents in which the word is present. We consider one occurrence if the term is present in the document at least once, we do not need to know the number of times the term is present.


    df(t) = occurrence of t in N documents


### Inverse Document Frequency

IDF is the inverse of the document frequency which measures the informativeness of term t. When we calculate IDF, it will be very low for the most occurring words such as stop words (because they are present in almost all of the documents, and N/df will give a very low value to that word). This finally gives what we want, a relative weightage.

    idf(t) = N/df

Now there are few other problems with the IDF, when we have a large corpus size say N=10000, the IDF value explodes. So to dampen the effect we take the log of IDF.

    idf(t) = log(N/(df + 1))


    tf-idf(t, d) = tf(t, d) * log(N/(df + 1))


## TF-IDF without sklearn
Note: This is not an efficient way of calculating tfidf vectors. This is just for demonstration purpose.


In [ ]:
N = len(documents)
vocab_lenght = len(vocab)
token2id = {word:index for index, word in enumerate(vocab)}


def normalize(X, norm='L2'):
    if norm == "L2":
        row_norm = np.sqrt((X * X).sum(axis=1))
        X = (X.T / row_norm).T
        return X

tf_idf = np.zeros((N,vocab_lenght))
tf_mat = np.zeros((N,vocab_lenght))
for document_index, document in enumerate(documents):
    tokens = document.split()
    #print(f"tokens: {tokens}")
    for token in tokens:
        token_id = token2id[token]

        tf = sum([1 for token_ in tokens if token_ == token])
        tf_mat[document_index][token_id] = tf
        df = sum([1 for document_ in documents if token in document_.split()])
        idf = math.log((N+1)/(df + 1)) + 1
        #print(f"token: {token}, tf: {tf}, df: {df}, idf: {idf}, tf_idf: {tf * idf}")
        
        
        tf_idf[document_index][token_id] = tf * idf
    #print("*"*100)

tf_idf = normalize(tf_idf)
print(f"Count Matrix: \n{tf_mat}\n")
print(f"TF-IDF Matrix: \n{tf_idf}\n")

Count Matrix: 
[[0. 1. 1. 1. 0. 0. 1. 0. 1.]
 [0. 2. 0. 1. 0. 1. 1. 0. 1.]
 [1. 0. 0. 1. 1. 0. 1. 1. 1.]
 [0. 1. 1. 1. 0. 0. 1. 0. 1.]]

TF-IDF Matrix: 
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]



## TF-IDF with sklearn

*TfidfVectorizer is equilent of applying CountVectorizer and then TfidfTransformer*

### TfidfVectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

In [ ]:
vectorizer = TfidfVectorizer(smooth_idf=True)
tf_idf = vectorizer.fit_transform(documents)

In [ ]:
print(f"TF-IDF Matrix: \n{tf_idf.toarray()}\n")

TF-IDF Matrix: 
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]



### CountVectorizer + TfidfTransformer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

In [ ]:
pipe = Pipeline([
    ('count', CountVectorizer(vocabulary=vocab)),
    ('tfid', TfidfTransformer())
]).fit(documents)
    

In [ ]:
count_vector = pipe['count'].transform(documents).toarray()
tf_idf = pipe.transform(documents).toarray()

In [ ]:
print(f"Count Matrix: \n{count_vector}\n")
print(f"TF-IDF Matrix: \n{tf_idf}\n")

Count Matrix: 
[[0 1 1 1 0 0 1 0 1]
 [0 2 0 1 0 1 1 0 1]
 [1 0 0 1 1 0 1 1 1]
 [0 1 1 1 0 0 1 0 1]]

TF-IDF Matrix: 
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]

